In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import re
import random

In [2]:
test_set = pd.read_csv("data/test_set_VU_DM.csv")
len(test_set)

4959183

In [3]:
df = pd.read_csv("data/training_set_VU_DM.csv")
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [4]:
df.srch_id.nunique()

199795

In [5]:
df[df["srch_id"] == 1]["prop_id"]

0        893
1      10404
2      21315
3      27348
4      29604
5      30184
6      44147
7      50984
8      53341
9      56880
10     59267
11     59526
12     68914
13     74474
14     81437
15     85728
16     88096
17     88127
18     88218
19     89073
20     95166
21     95307
22     97247
23    107872
24    111000
25    111106
26    114766
27    122844
Name: prop_id, dtype: int64

In [6]:
example = df[df["srch_id"] == 1]
h = example.head(2)

In [7]:
df.visitor_hist_starrating.count()

251866

In [8]:
df.srch_id.value_counts()

244169    38
149527    38
250956    38
223395    38
199641    37
          ..
258512     5
124354     5
247392     5
326283     5
104426     5
Name: srch_id, Length: 199795, dtype: int64

In [9]:
df.date_time.value_counts()

2012-11-02 14:04:51    95
2012-12-26 10:59:30    87
2013-05-15 18:39:51    70
2012-11-23 07:28:55    69
2013-06-16 10:43:33    68
                       ..
2012-12-26 13:52:51     5
2013-02-04 21:19:58     5
2012-12-31 19:43:57     5
2012-11-07 07:17:20     5
2013-04-05 23:05:39     5
Name: date_time, Length: 198615, dtype: int64

In [10]:
# Values that are missing in the test set
set(df.columns) - set(test_set.columns)

{'booking_bool', 'click_bool', 'gross_bookings_usd', 'position'}

In [11]:
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [12]:
df.describe()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
count,4.958347e+06,4.958347e+06,4.958347e+06,251866.000000,252988.000000,4.958347e+06,4.958347e+06,4.958347e+06,4.950983e+06,4.958347e+06,...,96174.000000,315348.000000,356422.000000,138515.000000,1.916654e+06,1.987503e+06,614730.000000,4.958347e+06,138390.000000,4.958347e+06
mean,1.663666e+05,9.953133e+00,1.753405e+02,3.374334,176.022659,1.739739e+02,7.007918e+04,3.180525e+00,3.777777e+00,6.346994e-01,...,17.250473,0.145969,0.083202,19.433267,-6.089936e-02,9.962752e-03,22.430384,4.474858e-02,386.283316,2.791051e-02
std,9.611223e+04,7.646890e+00,6.591625e+01,0.692519,107.254493,6.834525e+01,4.060992e+04,1.051024e+00,1.050329e+00,4.815144e-01,...,31.160313,0.578202,0.316722,54.370221,4.691723e-01,2.029142e-01,895.965854,2.067514e-01,821.190577,1.647165e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.410000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000,0.000000e+00,0.000000,0.000000e+00
25%,8.293600e+04,5.000000e+00,1.000000e+02,2.920000,109.810000,1.000000e+02,3.501000e+04,3.000000e+00,3.500000e+00,0.000000e+00,...,6.000000,0.000000,0.000000,7.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,124.000000,0.000000e+00
50%,1.665070e+05,5.000000e+00,2.190000e+02,3.450000,152.240000,2.190000e+02,6.963800e+04,3.000000e+00,4.000000e+00,1.000000e+00,...,11.000000,0.000000,0.000000,12.000000,0.000000e+00,0.000000e+00,11.000000,0.000000e+00,218.400000,0.000000e+00
75%,2.497240e+05,1.400000e+01,2.190000e+02,3.930000,213.490000,2.190000e+02,1.051680e+05,4.000000e+00,4.500000e+00,1.000000e+00,...,18.000000,1.000000,0.000000,20.000000,0.000000e+00,0.000000e+00,17.000000,0.000000e+00,429.790000,0.000000e+00
max,3.327850e+05,3.400000e+01,2.310000e+02,5.000000,1958.700000,2.300000e+02,1.408210e+05,5.000000e+00,5.000000e+00,1.000000e+00,...,1620.000000,1.000000,1.000000,9900.000000,1.000000e+00,1.000000e+00,149400.000000,1.000000e+00,159292.380000,1.000000e+00


In [13]:
len(df)

4958347

In [14]:
subset = df.head(100)
subset = subset.fillna(0)
subset.head(1)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


In [15]:
y = np.array(subset.booking_bool)
del subset["booking_bool"]
del subset["date_time"]
X = np.array(subset.values)

In [16]:
X

array([[  1.,  12., 187., ...,   0.,   0.,   0.],
       [  1.,  12., 187., ...,   0.,   0.,   0.],
       [  1.,  12., 187., ...,   0.,   0.,   0.],
       ...,
       [ 11.,   5., 219., ...,  10.,   0.,   0.],
       [ 11.,   5., 219., ...,  11.,   0.,   0.],
       [ 11.,   5., 219., ...,   0.,   0.,   0.]])

In [33]:
def SMOTE(X_train, y_train, mClass=1, k=3, ratio=.5, us=0):
    
    '''
    mClass = Binary minority class value (1 in case of fraud in 'simple_journal')
    X_train = training set 
    y_train = target values
    k = number of nearest neighbors k for SMOTEing
    Ratio = Desired ratio minority class relative to size of ORIGINAL majority class (50 by default)
    us = Undersampling ratio (1 =  * 1/2, 2 = * 1/4, etc.)

    '''    
        
    if type(k) != int or type(mClass) != int:
        raise Exception('Invalid input: "int" is required')
    elif type(X_train) != np.ndarray or type(y_train) != np.ndarray:
        raise Exception('Invalid input: "numpy.array" is required')
    elif k < 1 or ratio <= 0:
        raise Exception('Input value is too low')
        
    # Preprocess training data to obtain minority class
    # Extract all minority class values
    
    y_train = np.reshape(y_train, (len(y_train),1))    
    to_SMOTE = np.hstack((X_train, y_train))
          
    smote = np.zeros((len(X_train[0])+1,), dtype=int)
    indices =[]
    counter = 0
    for i in to_SMOTE:
        counter += 1
        if i[-1:] == mClass:
            smote = np.vstack((smote, i))
            indices.append(counter)
            
    to_SMOTE = np.delete(to_SMOTE, (indices), axis = 0)               
    smote = np.delete(smote, (0), axis=0)
    
    if us != 0:
        to_SMOTE = shuffle(to_SMOTE)
        u_sample = int(round((len(to_SMOTE)) * (1/(2**us))))
        under_sampling = to_SMOTE[0:u_sample]
    else:
        under_sampling = to_SMOTE

    
    ### --- ACTUAL SMOTING STARTS HERE ---
    # Calculate amount of artificial datapoints
    # Prepare array and counter
    
    P = ratio
    dp = int(round(((P*len(X_train))/(1-P)-len(smote))))
    N = int(dp/len(smote))
    data_ = N * len(smote)
    numattrs = len(smote[0])
    newindex = 0
    Synthetic =  np.zeros((data_, numattrs))
    

    def Populate(N, i, nnarray):
        global newindex
        while N != 0:
            nn = random.randint(1,k+1)
            for attr in range(numattrs):
                delta = random.uniform(0,1)
                dif = smote[nn][attr] - smote[i][attr]
                Synthetic[newindex][attr] = int(round((smote[i][attr] + (delta * dif))))
                # TRY choosing features of 1 of 3 neighbors
     
            newindex += 1
            N = N -1
    
    for i in range(len(smote)):
        nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(smote)
        print(smote[i].reshape(1, -1))
        print(nbrs.kneighbors(smote[i].reshape(1, -1)))
        distances, nnarray = nbrs.kneighbors(smote[i].reshape(1, -1))
        Populate(N, i, nnarray)
        
    ### --- ACTUAL SMOTEING ENDS HERE ---
  
    ## Convert to get the desired output in test and training data
    
    # Merge the generated and original datapoints together
    result = np.vstack((Synthetic, smote))
    
    # Prepare X_train and Y_train
    X_train_ = np.vstack((result, under_sampling))
    X_train_ = shuffle(X_train_)
    y_train = X_train_[:,-1].reshape((len(X_train_))).astype('int')
    X_train = np.delete(X_train_, len(X_train_[0])-1, axis=1)
    
    # Remove TomekLinks 
    tl = TomekLinks()
    X_train, y_train = tl.fit_sample(X_train, y_train)
    
    ratio_smote = int(round((len(result)/(len(result)+len(under_sampling)))*100))
    ratio_major = int(round(len(under_sampling)/(len(result)+len(under_sampling))*100))
    
    print("Final Data length \n-------------------------------------------")
    print("Minority Class: " + str(len(result)) + ", original: " + str(len(indices)) + 
          "\nMajority Class: " + str(len(under_sampling)) + ", original: " + str(len(to_SMOTE))+
         "\nTotal size X_train: " + str(len(X_train)))
    print()
    print("Final ratio \n--------------------------------------------")
    print("New ratio: " + str(ratio_smote) + "% Minority Class - " + str(ratio_major) + ("% Majority Class"))
    print()
    print()
    print()

    return X_train, y_train

In [34]:
# use over and undersampling
# Use smoothing to create artifical data point
# SMOTE does this

newindex = 0
SMOTE(X, y)

[[ 1.0000e+00  1.2000e+01  1.8700e+02  0.0000e+00  0.0000e+00  2.1900e+02
   6.8914e+04  2.0000e+00  3.0000e+00  1.0000e+00  2.2000e+00  2.0600e-02
   4.4400e+00  1.3000e+01  1.0089e+02  0.0000e+00  2.3246e+04  1.0000e+00
   0.0000e+00  4.0000e+00  0.0000e+00  1.0000e+00  1.0000e+00  0.0000e+00
   0.0000e+00  1.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  1.1000e+01  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00 -1.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
   0.0000e+00  1.1000e+01  1.0000e+00  1.1429e+02  1.0000e+00]]


ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 4

In [ ]:
correlations = df.corr()

In [ ]:
correlations["booking_bool"]

In [ ]:
clicked = df[df["booking_bool"] == 1]
clicked.head()

In [ ]:
# Split date time only into time
# Use time series!
df.date_time.head(5)